In [40]:
import os
import sys
from dotenv import load_dotenv
from googleapiclient.discovery import build
from youtube_transcript_api import YouTubeTranscriptApi

In [41]:
load_dotenv()
api_key = os.getenv("GCP_API_KEY")
channel_name = "@hubermanlab"

In [42]:
class YouTubeHelper:
    def __init__(self, api_key):
        self.api_key = api_key
        self.youtube = build('youtube', 'v3', developerKey=api_key)

    def get_channel_id(self, channel_name):
        request = self.youtube.search().list(
            q=channel_name,
            type='channel',
            part='id',
            maxResults=1
        )
        response = request.execute()
        if response['items']:
            return response['items'][0]['id']['channelId']
        else:
            return None

    def get_video_ids(self, channel_id):
        channel_response = self.youtube.channels().list(
            id=channel_id,
            part='contentDetails'
        ).execute()
        upload_playlist_id = channel_response['items'][0]['contentDetails']['relatedPlaylists']['uploads']
        videos = []
        next_page_token = None
        while True:
            playlist_response = self.youtube.playlistItems().list(
                playlistId=upload_playlist_id,
                part='contentDetails',
                maxResults=50,
                pageToken=next_page_token
            ).execute()
            videos += [item['contentDetails']['videoId'] for item in playlist_response['items']]
            next_page_token = playlist_response.get('nextPageToken')
            if next_page_token is None:
                break
        return videos

    def get_transcript(self, video_id):
        try:
            transcript = YouTubeTranscriptApi.get_transcript(video_id)
            output = ''
            for x in transcript:
                sentence = x['text']
                output += f'{sentence} '
            return transcript
        except:
            return None

In [33]:
yt_helper = YouTubeHelper(api_key)

In [35]:
channel_id = yt_helper.get_channel_id(channel_name)

In [36]:
video_ids = yt_helper.get_video_ids(channel_id)

In [38]:
example = video_ids[0]